# Supervised Learning

## 2) Logistic Regression (classification)

### 1-1) Logistic Regression from scratch

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# Step 1. Data Creation

## data ( height and weight of  adults and children)

In [ ]:
# The number of data to be created
m = 100

In [ ]:
adult_height = np.random.normal(175, 5, [m, 1])
adult_weight = np.random.normal(70, 5, [m, 1])

adult_dataset = np.concatenate( (adult_weight, adult_height) , axis = 1)

print(adult_dataset.shape)
print(adult_dataset[:5])

In [ ]:
child_height = np.random.normal(120, 5, [m, 1])
child_weight = np.random.normal(30, 5, [m, 1])

child_dataset = np.concatenate( (child_weight, child_height) , axis = 1)

print(child_dataset.shape)
print(child_dataset[:5])

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(adult_dataset[:,0], adult_dataset[:,1], c=np.array([(1, 1, 0)]))
ax1.scatter(child_dataset[:,0], child_dataset[:,1], c=np.array([(0, 1, 0)]))
plt.title("The distribution of created dataset ")
plt.xlabel("height x2")
plt.ylabel("weight x1")
plt.show()

## Label

In [ ]:
# 각각의 클레스에 대한 레이블을 만들어줍니다.
# adult는 1로, child는 0으로 지정합니다.
# binary classification은 내가 찾는 클래스(1)이냐 아니냐(0)를 분류합니다.
# 즉 우리가 학습시킬 모델은 성인과 성인이 아닌 데이터를 분류하는 모델입니다.

adult_label = np.ones( shape=[m, 1] )
child_label = np.zeros( shape=[m, 1] )
label = np.concatenate( (adult_label, child_label) )
print('label의 shape' , label.shape)
print(label[:10])

## Data + Label

In [ ]:
total_dataset = np.concatenate((adult_dataset , child_dataset))
total_dataset = np.concatenate( (total_dataset, label), axis = 1  )

np.random.shuffle(total_dataset) # Shuffle dataset
print(total_dataset[:10])
print(total_dataset.shape)

## simple scaling of dataset

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(total_dataset[:,:2])
print(scaler.mean_)


In [ ]:
total_dataset[:,:2] = scaler.transform(total_dataset[:,:2])

In [ ]:
total_dataset.shape

fig = plt.figure()
ax1 = fig.add_subplot(111)
color = [(1*i, 1, 0) for i in total_dataset[:,2] ]
ax1.scatter(total_dataset[:,0], total_dataset[:,1], c = color)
plt.show()

# Step 2. Model definition

In [ ]:
def linear(x, w, b):
    pred = np.matmul(x, w.T) + b
    return pred

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [ ]:
def hypothesis(x,w,b):
    return sigmoid(linear(x, w, b))

In [ ]:
def cost(x, w, b, y):
    loss = -y*linear(x, w, b) + np.log(1 + np.exp(linear(x, w, b))) 
    #-ylog(h(x))-(1-y)log(1-h(x)) =  -y*(wx+b) + log(1+exp(wx+b))
    cost = (1/m)*np.sum(loss) # mean of losses
    return cost

In [ ]:
def derivative(x, w, b, y):
    dw = (1/m)* np.sum( x*(hypothesis(x, w, b)-y) , axis = 0) # 1 x w_dim
    # (1/m)*x(h(x)-y)
    db = (1/m)* np.sum( hypothesis(x, w, b)-y, axis = 0 )
    # (1/m)*(h(x)-y)
    return dw, db

In [ ]:
def update(x, w, b, y, alpha):
    w = w - alpha*(derivative(x, w, b, y)[0]) # w := w - alpha * dw
    b = b - alpha*(derivative(x, w, b, y)[1]) # b := b - alpha * db
    return w, b

# Step 3. Training model

In [ ]:
# initialization of weights and bias
w = np.array([[3.0, -3.0]])
b = -1.345

x = total_dataset[:, :2] # x1, x2
y = total_dataset[:, 2:3] # label

## 아래 두 개 cell을 반복하여 실행하며, 선형 분류기의 위치 변화를 관찰해보세요.

In [ ]:
def visualize(w, b):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    color = [(1*i, 1, 0) for i in total_dataset[:,2] ]
    ax1.scatter(total_dataset[:,0], total_dataset[:,1], c = color)

    p1 = np.array([0, -b/w[0, 1]])
    p2 = np.array([-b[0]/w[0, 0], 0]) # b has the shape as [1]
    coefficients = np.polyfit([p1[0], p2[0]], [p1[1][0], p2[1]], 1)  
    polynomial = np.poly1d(coefficients)
    x_axis = np.linspace(-2.0, 2.0)
    y_axis = polynomial(x_axis)
    ax1.set_ylim(-2.0, 2.0)
    ax1.plot(x_axis, y_axis)

    plt.show()

In [ ]:
for i in range(3001):
    w, b = update(x, w, b, y, alpha=0.005)
    if i%500 == 0:
      print('cost = ', cost(x, w, b, y))
      visualize(w, b)

In [ ]:
from sklearn.metrics import accuracy_score
preds = hypothesis(x,w,b)
preds = (preds>0.5).astype(np.int32)
print(preds[:10])

In [ ]:
y[:10]

In [ ]:
print("정확도 = ", accuracy_score(np.squeeze(preds), np.squeeze(y)))

## 라이브러리 사용

In [ ]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(x,y)

In [ ]:
LR.predict(x)

# 이런 문제는 어떨까요?

In [ ]:
# 임의의 클래스 A와 클래스 B를 생성함.
m = 200
class_A = np.random.normal(0, 1, [m, 2])
label_A = np.ones([m, 1])

class_B_x = np.random.normal(0, 2, [m, 1])
class_B_y = 0.5*class_B_x**2 - 3

class_B = np.concatenate([class_B_x, class_B_y], axis = 1)
label_B = np.zeros([m, 1])

In [ ]:
#데이터 분포를 볼까요?

fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.set_title('nonlinear distribution')
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')

ax1.set_xlim([-4,4])
ax1.set_ylim([-4,4])

ax1.scatter(class_A[:,0], class_A[:,1], c='red')

ax1.scatter(class_B[:,0], class_B[:,1])
plt.show()

## 다른 종류의 분류

In [ ]:
import tensorflow as tf

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
#데이터의 shape을 볼까요?
print(x_train.shape)

In [ ]:
# 60000개 중 첫번째 데이터를 봅시다.
img = x_train[0]
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
# 값도 출력해봅시다.
print(img)

In [ ]:
# 값도 출력해봅시다.
vectorized = np.reshape(img, [28*28])

In [ ]:
vectorized.shape

In [ ]:
#이런 데이터도 잘 분류될 수 있을까?
print(vectorized)